In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
ind = "network_weather_2-*"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 1},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 2}
                        }
                     }
                  }
              }
        }
  }
}

In [3]:
response = es.search(index=ind, body=my_query, request_timeout=600)

In [4]:
#print(response['aggregations']['type_counts']['buckets'])
toDelete={}

In [5]:
types = response['aggregations']['type_counts']['buckets']

for t in types:
    print(t['key'])
    if t['key'] != 'packet_loss_rate': continue
    for s in t['src_counts']['buckets']:
        for d in s['dest_counts']['buckets']:
            print ('s: ',s['key'],'d: ',d['key'])
            
        
            myscroll={
            "fields": ["timestamp", "delay_mean","throughput","packet_loss"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src": {"value": s['key'] } } },
                            {"term": {"dest": {"value": d['key'] } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": { "timestamp": { "order": "asc" }}
            }
            
            scroll = scan(es, query=myscroll, index=ind, scroll='10s')
            oldtype=""
            oldtime=""
            counter=0
            for res in scroll:
                newtime=res['fields']['timestamp']
                if newtime==oldtime:
                    toDelete[res['_id']]=res['_index']
                oldtime=newtime
                counter+=1
            print ("records:", counter)
            print ('\tto delete:',len(toDelete))

packet_loss_rate
s:  85.122.31.74 d:  134.79.118.72


KeyboardInterrupt: 